In [1]:
# Import Dependencies
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Business Insider Commodities URL to scrape
url = 'https://markets.businessinsider.com/commodities'
browser.visit(url)
html = browser.html

In [4]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Search through each table and extract the data from the rows of each table
data=[]
tables = soup.find_all('table')
for table in tables:
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values

In [6]:
# Review the data
print(data)

[['Orange Juice', '4.12%', '2.19 USD'], ['Nickel', '2.80%', '22,150.00 USD'], ['Cocoa', '1.33%', '1,976.00 GBP'], ['Live Cattle', '0.95%', '1.52 USD'], ['Coal', '0.75%', '270.00 USD'], ['Gold', '1,645.28', '-0.27%', '-4.42', 'USD per Troy Ounce', '07:41:00 AM'], ['Palladium', '1,965.24', '-0.49%', '-9.76', 'USD per Troy Ounce', '07:41:01 AM'], ['Platinum', '916.50', '-1.35%', '-12.50', 'USD per Troy Ounce', '07:33:00 AM'], ['Silver', '18.91', '-1.97%', '-0.38', 'USD per Troy Ounce', '07:33:00 AM'], ['Natural Gas (Henry Hub)', '5.16', '-0.33%', '-0.02', 'USD per MMBtu', '07:23:00 AM'], ['Ethanol', '2.16', '0.05%', '0.00', 'per Gallon', '10/24/2022'], ['Heating Oil', '102.50', '-1.52%', '-1.59', 'USD per 100 Liter', '07:23:00 AM'], ['Coal', '270.00', '0.75%', '2.00', 'per Ton', '10/20/2022'], ['RBOB Gasoline', '2.70', '-1.71%', '-0.05', 'per Gallone', '07:20:00 AM'], ['Uranium', '0.85', '%', 'per 250 Pfund U308', '9/28/2022'], ['Oil (Brent)', '91.81', '-1.75%', '-1.64', 'USD per Barrel',

In [7]:
browser.quit()

In [8]:
# Iterate through the rows and get the data for the four commodites we are plotting
commodities = []
for row in data:
    if row[0] == 'Gold':
        commodities.append(row)
    if row[0] == 'Nickel':
        commodities.append(row)
    if row[0] == 'Aluminium':
        commodities.append(row)
    if row[0] == 'Iron Ore':
        commodities.append(row)

In [9]:
print(commodities)

[['Nickel', '2.80%', '22,150.00 USD'], ['Gold', '1,645.28', '-0.27%', '-4.42', 'USD per Troy Ounce', '07:41:00 AM'], ['Aluminium', '2,181.25', '-1.53%', '-33.96', 'USD per Ton', '10/24/2022'], ['Iron Ore', '94.51', '-0.37%', '-0.35', 'per Dry Metric Ton', '10/24/2022'], ['Nickel', '22,150.00', '2.80%', '602.50', 'USD per Ton', '10/24/2022']]


In [10]:
# Create the pandas DataFrame
df = pd.DataFrame(commodities, columns = ['Commodity', 'Price', '% Change','+/-','Unit','Date'])

In [11]:
#Remove row of partial data (which comes from the ynamic table at the top of the webpage)
df.dropna(axis=0,how='any',inplace=True)
df

,Commodity,Price,% Change,+/-,Unit,Date
1,Gold,"1,645.28",-0.27%,-4.42,USD per Troy Ounce,07:41:00 AM
2,Aluminium,"2,181.25",-1.53%,-33.96,USD per Ton,10/24/2022
3,Iron Ore,94.51,-0.37%,-0.35,per Dry Metric Ton,10/24/2022
4,Nickel,"22,150.00",2.80%,602.50,USD per Ton,10/24/2022


In [12]:
#Export dataframe to CSV
df.to_csv('resources/commodity_price.csv', index=False)